In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output

import dash_table as dt
import base64

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

from AnimalShelter import AnimalShelter


app = JupyterDash('SimpleExample')
###########################
# Data Manipulation / Model
###########################


username = "hunt2"
password = "Password"
shelter = AnimalShelter(username, password)



cols = ["id","age_upon_outcome","animal_id","animal_type","breed","color",
       "date_of_birth","datetime","monthyear","name","outcome_subtype","outcome_type",
       "sex_upon_outcome","location_lat","location_long","age_upon_outcome_in_weeks"]
data = shelter.read({})

df=pd.DataFrame.from_records(data, columns =cols)





#########################
# Dashboard Layout / View
#########################

logo ='Grazioso Salvare Logo.png'
encode_image = base64.b64encode(open(logo, 'rb').read()).decode('ascii')



app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A(html.Img(src='data:image/png;base64,{}'.format(encode_image),
                   style={'width':'100px','height':'100px'})),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Erwin Gove'))),
    html.Hr(),

    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label':'Water Rescue','value':'Water'},
                {'label': 'Mountain/Wilderness Rescue', 'vaue':'Mountain'},
                {'label': 'Disaster Rescue',"value":"Disaster"},
                {'label':"Reset","value":"reset"}
            ],
            value="reset",
            labelStyle={'display':'inline-block'})),
       
    html.Hr(),
    
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        row_selectable="single",
        page_action="native",
        page_current=0,
        page_size=10,      
    ),
        
    html.Br(),
    html.Hr(),
              
    html.Div(className='row',
            style={'display':'flex'},
            children=[
                html.Div(
                    dcc.Graph(id="graph-id")),
                
                html.Div(
                    id='map-id',
                    className='col s12 m6',)
            ]),
],style={'textAlign':'center'})

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it


@app.callback([Output('datatable-id','data'),
             Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filterType):
    

    cols = ["id","age_upon_outcome","animal_id","animal_type","breed","color",
       "date_of_birth","datetime","monthyear","name","outcome_subtype","outcome_type",
       "sex_upon_outcome","location_lat","location_long","age_upon_outcome_in_weeks"]
 
    
    if filterType =="Water":
        df=pd.DataFrame(list(shelter.read({'$and':[
            {'$or':[{'breed': 'Labrador Retriever Mix'},
                    {'breed': 'Chesa Bay Retr Mix'},
                    {'breed': 'Newfoundland Mix'},
                    {'breed':'Newfoundland/Labrador Retriever'},
                    {'breed':'Labrador Retriever/Newfoundland'},
                    {'breed':'NewfoundLand/Great Pyrenees'},
                    {'breed':'Newfoundland/Austrailian Cattle Dog'},
                    {'breed':'Labrador Retriever/Chesa Bay Retr'}]},
            {'sex_upon_outcome':"Intact Female"},
            {'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}}]}),
                                                  columns =cols))
        
    elif filterType == "Mountain":
        df=pd.DataFrame(list(shelter.read({'$and':[
            {'$or':[{'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Rottweiler'},
                    {'breed': 'Siberian Husky'}]},
            {'sex_upon_outcome':'Intact Male'},
            {'age_upon_outcome_in_weeks':{'$gte':26,'$lte':156}},
        ]}),
                                                  columns =cols))
        
    elif filterType == "Disaster":
        df=pd.DataFrame(list(shelter.read({'$and':[
            {'$or':[{'breed': 'Golden Retriever'},
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Rottweiler'},
                    {'breed': 'Bloodhound'}]},
            {'sex_upon_outcome':'Intact Male'},
            {'age_upon_outcome_in_weeks':{'$gte':20,'$lte':300}},
        ]}),
                                                  columns =cols))
    
    elif filterType == "reset":
        df = pd.DataFrame.from_records(shelter.read({}), columns =cols)
    
    columns=[{"name":i,"id":i,"deletable":False,"selectable":True}for i in df.columns]
    
    
    data=df.to_dict('records')
    
    return (data,columns)

    
   

@app.callback(
    Output("graph-id","figure"),
    [Input('datatable-id',"data")])
def update_graphs(viewData):
   
    dff = pd.DataFrame.from_dict(viewData)
    out = px.pie(dff, names='breed')
        
    return out 
       
""" 
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
"""

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id',"derived_viewport_selected_rows")
    ])
def update_map(viewData, selectedRow):
    if viewData is None:
        return []
    selected = 0
    if selectedRow is not None and len(selectedRow)>0:
        selected = selectedRow[0]
    


    dff = pd.DataFrame.from_dict(viewData)

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, 
               center=[dff.iloc[0,13],dff.iloc[0,14]],
               zoom=10,
               children=[
                dl.TileLayer(id="base-layer-id"),
            
                # popup thing displayin animal name on map
                dl.Marker(position=[dff.iloc[selected,13],
                                    dff.iloc[selected,14]],
                          children=[
                            dl.Tooltip(dff.iloc[selected,9]),
                            dl.Popup([
                                html.H1(dff.iloc[selected,9]),
                                html.P(dff.iloc[selected,9])
                ])
            ])
        ])
    ]


app

